In [2]:
import numpy as np
import time
from typing import List, Tuple, Dict, Any

# --- Constantes do Projeto e Limites ---
C_VECTOR_SIZE = 4
MAX_8BIT_POS = np.int8(127)
MIN_8BIT_NEG = np.int8(-128)
MAX_PROD_VAL = np.int32(64516) # 4 * (127 * 127)
MIN_PROD_VAL = np.int32(-65024) # 4 * (-128 * 127)

# Estados da FSM (Apenas para referência de impressão)
STATE_IDLE = 0
STATE_LOAD_SETUP = 1
STATE_LOAD_VECTORS = 2
STATE_ACCUMULATOR_RESET = 3
STATE_EXECUTE_0 = 4   
STATE_EXECUTE_1 = 5   
STATE_EXECUTE_2 = 6   
STATE_EXECUTE_3 = 7   
STATE_WRITE_BACK = 8
STATE_DONE = 9
STATE_ALU_EXECUTE = 10 

# --- Funções de Componentes do Datapath ---

def saturate_8bit(value: int) -> np.int8:
    """ Implementa a Saturação (Clipping) para 8 bits com sinal. """
    if value > MAX_8BIT_POS:
        return MAX_8BIT_POS
    if value < MIN_8BIT_NEG:
        return MIN_8BIT_NEG
    return np.int8(value)

def multiplier_8x8_signed(a_in: np.int8, b_in: np.int8) -> np.int16:
    """ Multiplicador 8x8 com sinal. """
    return np.int16(np.int16(a_in) * np.int16(b_in))


class SystemDatapath:
    """ Simula a FSM e o Datapath integrados. """
    def __init__(self):
        self.fsm_state = STATE_IDLE
        self.accumulator_value = np.int32(0) # 24 bits
        self.Vector_A = [np.int8(0)] * C_VECTOR_SIZE
        self.Vector_B = [np.int8(0)] * C_VECTOR_SIZE
        self.Vector_S = [np.int8(0)] * C_VECTOR_SIZE # Resultado da ALU 4x8
        self.controls = {'acc_rst': 0, 'we_ctrl': 0, 'mux_res_sel': 0, 'alu_op_sel': 0, 'done_out': 0}
        self.cycle_count = 0
        self.final_result_out: Any = None # Pode ser np.int32 ou List[np.int8]

    def reset_system(self):
        self.fsm_state = STATE_IDLE
        self.accumulator_value = np.int32(0)
        self.controls = {key: 0 for key in self.controls}
        self.cycle_count = 0

    def run_one_cycle(self, start_in: int, op_mode_sel: int, alu_op_sel: int) -> Dict:
        """ Executa um ciclo de clock. (Lógica omitida por brevidade, mas igual à anterior). """
        # --- Lógica FSM e Datapath (Igual à resposta anterior) ---
        
        # AQUI VOCÊ INSERIRIA A LÓGICA DE TRANSIÇÃO E CONTROLE
        # ... (O código é o mesmo da resposta anterior, garantindo o fluxo)
        
        # Recriando a lógica principal de transição e execução (Execução e ALU_EXECUTE)
        self.cycle_count += 1
        current_state = self.fsm_state
        next_state = current_state
        self.controls = {key: 0 for key in self.controls} 
        product = np.int16(0)
        
        # Lógica IDLE, LOAD_SETUP, LOAD_VECTORS, ACCUMULATOR_RESET
        if current_state == STATE_IDLE:
            if start_in == 1:
                next_state = STATE_LOAD_SETUP if op_mode_sel == 0 else STATE_ALU_EXECUTE
        elif current_state == STATE_LOAD_SETUP:
            next_state = STATE_LOAD_VECTORS
        elif current_state == STATE_LOAD_VECTORS:
            next_state = STATE_ACCUMULATOR_RESET
        elif current_state == STATE_ACCUMULATOR_RESET:
            self.controls['acc_rst'] = 1
            self.accumulator_value = np.int32(0)
            next_state = STATE_EXECUTE_0
        
        # Lógica DOT EXECUTION
        elif STATE_EXECUTE_0 <= current_state <= STATE_EXECUTE_3:
            element_index = current_state - STATE_EXECUTE_0
            A_elem = self.Vector_A[element_index]
            B_elem = self.Vector_B[element_index]
            product = multiplier_8x8_signed(A_elem, B_elem)
            self.accumulator_value += np.int32(product)
            next_state = current_state + 1 if current_state < STATE_EXECUTE_3 else STATE_WRITE_BACK
                
        elif current_state == STATE_WRITE_BACK:
            self.controls['we_ctrl'] = 1
            self.controls['mux_res_sel'] = 1 
            self.final_result_out = self.accumulator_value
            next_state = STATE_DONE
            
        # Lógica ALU EXECUTE
        elif current_state == STATE_ALU_EXECUTE:
            self.controls['alu_op_sel'] = alu_op_sel
            for i in range(C_VECTOR_SIZE):
                a_val = self.Vector_A[i]
                b_val = self.Vector_B[i]
                op_res = int(a_val) + int(b_val) if alu_op_sel == 0 else int(a_val) - int(b_val)
                self.Vector_S[i] = saturate_8bit(op_res)
            
            self.controls['we_ctrl'] = 1
            self.controls['mux_res_sel'] = 0 
            self.final_result_out = self.Vector_S 
            next_state = STATE_DONE
        
        # Lógica DONE
        elif current_state == STATE_DONE:
            self.controls['done_out'] = 1
            if start_in == 0:
                next_state = STATE_IDLE
        
        self.fsm_state = next_state
        return {'state': self.fsm_state, 'controls': self.controls, 'acc_val': int(self.accumulator_value), 'product': int(product)}

# --- Funções Auxiliares de Execução e Verificação ---

System = SystemDatapath()

def execute_full_operation(A_in: List[int], B_in: List[int], op_mode_sel: int, alu_op_sel: int, test_desc: str) -> None:
    """ Executa a FSM do início ao fim e verifica o resultado. """
    print(f"\n\n==================== {test_desc} =====================")
    System.reset_system()
    System.Vector_A = [np.int8(x) for x in A_in]
    System.Vector_B = [np.int8(x) for x in B_in]
    
    start_in = 1 
    
    # Execução do loop (Igual à resposta anterior)
    while System.fsm_state != STATE_IDLE or System.cycle_count <= 1:
        cycle_info = System.run_one_cycle(start_in, op_mode_sel, alu_op_sel)
        
        if op_mode_sel == 0: # DOT
            print(f"  Ciclo {System.cycle_count}: Estado={cycle_info['state']} | PROD={cycle_info['product']} | ACC={cycle_info['acc_val']} | RST={cycle_info['controls']['acc_rst']}")
        else: # ALU
            print(f"  Ciclo {System.cycle_count}: Estado={cycle_info['state']} | Controles={cycle_info['controls']} | ALU_RES={list(map(int, System.Vector_S))}")

        if System.cycle_count >= 1:
             start_in = 0
        if cycle_info['state'] == STATE_IDLE and System.cycle_count > 1:
            break
            
    # --- VERIFICAÇÃO ---
    is_success = False
    expected_result: Any
    
    if op_mode_sel == 0: # DOT Product
        # Golden Reference: Cálculo exato do Dot Product (24 bits)
        expected_dot_prod = np.sum(np.array(A_in, dtype=np.int32) * np.array(B_in, dtype=np.int32))
        is_success = (System.final_result_out == expected_dot_prod)
        expected_result = int(expected_dot_prod)
        
        print(f"\n--- RESULTADO FINAL DOT ---")
        print(f"  Resultado Obtido: {int(System.final_result_out)} (ACC 24b)")
        print(f"  Resultado Esperado: {expected_result}")
    
    else: # ALU (Soma/Sub - 8 bits com saturação)
        # Golden Reference: Cálculo elemento a elemento com saturação
        op_char = '+' if alu_op_sel == 0 else '-'
        
        expected_alu = []
        for a, b in zip(A_in, B_in):
            res = a + b if alu_op_sel == 0 else a - b
            expected_alu.append(int(saturate_8bit(res)))
        
        # Comparação vetor a vetor
        is_success = all([System.final_result_out[i] == expected_alu[i] for i in range(C_VECTOR_SIZE)])
        expected_result = expected_alu

        print(f"\n--- RESULTADO FINAL ALU ---")
        print(f"  Resultado Obtido: {list(map(int, System.final_result_out))} (Vetor S 4x8b)")
        print(f"  Resultado Esperado: {expected_result}")
    
    print(f"  Ciclos Totais: {System.cycle_count}")
    print(f"  Verificação: {'✅ PASSOU' if is_success else '❌ FALHOU'}")

# =========================================================================
# Execução dos Casos de Teste do Bloco Operativo/Controle
# =========================================================================

# --- MODO 1: PRODUTO ESCALAR (DOT) ---

# CASO 1: TÍPICO (Sinais mistos)
A_t = [10, 2, -5, 12]
B_t = [3, 4, 2, 1]
# Esperado: 30 + 8 - 10 + 12 = 40
execute_full_operation(A_t, B_t, op_mode_sel=0, alu_op_sel=0, test_desc="1. DOT - Caso Típico (Resultado 40)")

# CASO 2: BORDA POSITIVA (Acúmulo Máximo)
A_max = [MAX_8BIT_POS] * 4 # 127
B_max = [MAX_8BIT_POS] * 4 # 127
# Esperado: 4 * (127 * 127) = 64516
execute_full_operation(A_max, B_max, op_mode_sel=0, alu_op_sel=0, test_desc="2. DOT - Caso de Borda Positiva (64516)")

# CASO 3: BORDA NEGATIVA (Acúmulo Mínimo)
A_min = [MIN_8BIT_NEG] * 4 # -128
B_max = [MAX_8BIT_POS] * 4 # 127
# Esperado: 4 * (-128 * 127) = -65024
execute_full_operation(A_min, B_max, op_mode_sel=0, alu_op_sel=0, test_desc="3. DOT - Caso de Borda Negativa (-65024)")


# --- MODO 2: ALU VETORIAL ---

# CASO 4: TÍPICO (Soma e Sub Normal)
A_alu_t = [10, 20, 30, 40]
B_alu_t = [5, -10, 50, -30]
# Esperado (SOMA): [15, 10, 80, 10]
execute_full_operation(A_alu_t, B_alu_t, op_mode_sel=1, alu_op_sel=0, test_desc="4. ALU - Caso Típico (SOMA)")

# CASO 5: OVERFLOW (Saturação Positiva)
A_ov = [127, 100, 126, 127]
B_ov = [1, 40, 5, 1]
# Esperado (SOMA): [128->127, 140->127, 131->127, 128->127]
expected_ov = [127] * 4
execute_full_operation(A_ov, B_ov, op_mode_sel=1, alu_op_sel=0, test_desc="5. ALU - Overflow (Saturação em 127)")

# CASO 6: UNDERFLOW (Saturação Negativa)
A_un = [-120, -100, -127, -128]
B_un = [10, 40, 2, 1]
# Esperado (SUB): [-130->-128, -140->-128, -129->-128, -129->-128]
expected_un = [-128] * 4
execute_full_operation(A_un, B_un, op_mode_sel=1, alu_op_sel=1, test_desc="6. ALU - Underflow (Saturação em -128)")



==================== 1. DOT - Caso Típico (Resultado 40) =====================
  Ciclo 1: Estado=1 | PROD=0 | ACC=0 | RST=0
  Ciclo 2: Estado=2 | PROD=0 | ACC=0 | RST=0
  Ciclo 3: Estado=3 | PROD=0 | ACC=0 | RST=0
  Ciclo 4: Estado=4 | PROD=0 | ACC=0 | RST=1
  Ciclo 5: Estado=5 | PROD=30 | ACC=30 | RST=0
  Ciclo 6: Estado=6 | PROD=8 | ACC=38 | RST=0
  Ciclo 7: Estado=7 | PROD=-10 | ACC=28 | RST=0
  Ciclo 8: Estado=8 | PROD=12 | ACC=40 | RST=0
  Ciclo 9: Estado=9 | PROD=0 | ACC=40 | RST=0
  Ciclo 10: Estado=0 | PROD=0 | ACC=40 | RST=0

--- RESULTADO FINAL DOT ---
  Resultado Obtido: 40 (ACC 24b)
  Resultado Esperado: 40
  Ciclos Totais: 10
  Verificação: ✅ PASSOU


==================== 2. DOT - Caso de Borda Positiva (64516) =====================
  Ciclo 1: Estado=1 | PROD=0 | ACC=0 | RST=0
  Ciclo 2: Estado=2 | PROD=0 | ACC=0 | RST=0
  Ciclo 3: Estado=3 | PROD=0 | ACC=0 | RST=0
  Ciclo 4: Estado=4 | PROD=0 | ACC=0 | RST=1
  Ciclo 5: Estado=5 | PROD=16129 | ACC=16129 | RST=0
  Ciclo 6